## **Trabajo en clase: TOPIC MODELING**  
**Michael Pillaga**


### 1. Lectura del archivo CSV y visualización inicial  
Lee el archivo CSV que contiene los datos del podcast, como el `id`, el `guest`, el `title` y las transcripciones (`text`). Luego, muestra las primeras 5 filas del DataFrame para confirmar que los datos se cargaron correctamente.


In [11]:
import pandas as pd
df = pd.read_csv('./podcastdata_dataset.csv')
print(df.head())

   id            guest                    title  \
0   1      Max Tegmark                 Life 3.0   
1   2    Christof Koch            Consciousness   
2   3    Steven Pinker  AI in the Age of Reason   
3   4    Yoshua Bengio            Deep Learning   
4   5  Vladimir Vapnik     Statistical Learning   

                                                text  
0  As part of MIT course 6S099, Artificial Genera...  
1  As part of MIT course 6S099 on artificial gene...  
2  You've studied the human mind, cognition, lang...  
3  What difference between biological neural netw...  
4  The following is a conversation with Vladimir ...  


### 2. División del texto en bloques de oraciones  
Se define una función `agrupar_oraciones` para dividir el texto de cada transcripción en bloques de 200 oraciones (o el tamaño especificado). Luego, se aplica esta función a la columna `text` del DataFrame `df`.  
Los bloques generados se expanden en un nuevo DataFrame (`bloques_df`), manteniendo información relevante como el `id`, el `guest` y el `title` del episodio correspondiente.  
Finalmente, se muestra una vista previa de los primeros bloques generados.


In [12]:
def agrupar_oraciones(oraciones, bloque_size=200):
    bloques = []
    for i in range(0, len(oraciones), bloque_size):
        bloque = " ".join(oraciones[i:i+bloque_size])  # Concatenar oraciones del bloque
        bloques.append(bloque)
    return bloques

# Aplicamos la función y expandimos los bloques en un nuevo DataFrame
from itertools import chain

df['bloques'] = df['text'].apply(lambda texto: agrupar_oraciones(texto.split('.'), bloque_size=200))

bloques_df = pd.DataFrame(
    list(chain.from_iterable(df.apply(lambda row: [
        (row['id'], row['guest'], row['title'], bloque) for bloque in row['bloques']
    ], axis=1))),
    columns=['id', 'guest', 'title', 'bloque_texto']
)

print(bloques_df.head())


   id          guest          title  \
0   1    Max Tegmark       Life 3.0   
1   1    Max Tegmark       Life 3.0   
2   1    Max Tegmark       Life 3.0   
3   1    Max Tegmark       Life 3.0   
4   2  Christof Koch  Consciousness   

                                        bloque_texto  
0  As part of MIT course 6S099, Artificial Genera...  
1   The way they get AGI is building a quantum co...  
2   So there's, it's almost like a sea that's ris...  
3   And that kind of information helps to build t...  
4  As part of MIT course 6S099 on artificial gene...  


### 3. Tokenización de los bloques de texto  
Usamos `word_tokenize` de la librería NLTK para dividir cada bloque de texto (`bloque_texto`) en palabras o tokens individuales.  
La tokenización se aplica a cada fila del DataFrame `bloques_df` y los resultados se almacenan en una nueva columna llamada `tokens`.  
Se muestra una vista previa de los primeros bloques tokenizados, junto con el `id`, el `guest` y el `title` del episodio.


In [13]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

bloques_df['tokens'] = bloques_df['bloque_texto'].apply(word_tokenize)

print(bloques_df[['id', 'guest', 'title', 'tokens']].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Saitama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


   id          guest          title  \
0   1    Max Tegmark       Life 3.0   
1   1    Max Tegmark       Life 3.0   
2   1    Max Tegmark       Life 3.0   
3   1    Max Tegmark       Life 3.0   
4   2  Christof Koch  Consciousness   

                                              tokens  
0  [As, part, of, MIT, course, 6S099, ,, Artifici...  
1  [The, way, they, get, AGI, is, building, a, qu...  
2  [So, there, 's, ,, it, 's, almost, like, a, se...  
3  [And, that, kind, of, information, helps, to, ...  
4  [As, part, of, MIT, course, 6S099, on, artific...  


### 4. Entrenamiento del modelo Word2Vec  
Entrenamos un modelo Word2Vec utilizando las listas de tokens generadas en el paso anterior.  
- **`sentences=bloques_df['tokens']`**: El modelo se entrena con los tokens de cada bloque.  
- **`vector_size=100`**: Cada palabra será representada por un vector de 100 dimensiones.  
- **`window=5`**: El modelo considera un contexto de 5 palabras antes y después del término actual.  
- **`min_count=1`**: Se incluyen todas las palabras, incluso aquellas que aparecen solo una vez.  
El modelo aprende relaciones semánticas entre palabras basadas en el contexto dentro de los bloques de texto.


In [14]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=bloques_df['tokens'], vector_size=100, window=5, min_count=1, workers=4)


### 5. Cálculo de los embeddings para cada bloque de texto  
Para cada bloque de texto tokenizado, se genera un *embedding* que representa el significado promedio de todas las palabras del bloque.  
- **`obtener_embedding(bloque_tokens, modelo)`**: Obtiene los vectores de las palabras y calcula el promedio para generar un único vector representativo del bloque.  
- Si un bloque no tiene palabras conocidas por el modelo, se retorna un vector de ceros.  
Los resultados se almacenan en la columna `embeddings` del DataFrame, y se muestra una vista previa de los primeros bloques con sus respectivos embeddings.


In [15]:
import numpy as np

def obtener_embedding(bloque_tokens, modelo):
    vectores = [modelo.wv[word] for word in bloque_tokens if word in modelo.wv]
    if len(vectores) > 0:
        return np.mean(vectores, axis=0)  # Promediar los vectores
    else:
        return np.zeros(modelo.vector_size)

bloques_df['embeddings'] = bloques_df['tokens'].apply(lambda x: obtener_embedding(x, model))

print(bloques_df[['id', 'guest', 'title', 'embeddings']].head())


   id          guest          title  \
0   1    Max Tegmark       Life 3.0   
1   1    Max Tegmark       Life 3.0   
2   1    Max Tegmark       Life 3.0   
3   1    Max Tegmark       Life 3.0   
4   2  Christof Koch  Consciousness   

                                          embeddings  
0  [-0.49504837, -0.45915484, 0.1681212, 0.111881...  
1  [-0.52780443, -0.46918842, 0.10328533, 0.10727...  
2  [-0.5186225, -0.44692662, 0.13203557, 0.075770...  
3  [-0.48111993, -0.4711618, 0.21049581, 0.142526...  
4  [-0.42403653, -0.45269567, 0.1766183, 0.001626...  


### 6. Reducción de dimensionalidad usando PCA  
Se aplica **Análisis de Componentes Principales (PCA)** para reducir la dimensionalidad de los embeddings generados en el paso anterior.  
- **`n_components=50`**: Los vectores de embeddings originales se reducen a 50 dimensiones para optimizar la eficiencia computacional sin perder demasiada información.  
- **`pca.fit_transform()`**: Ajusta y transforma los embeddings para obtener una representación de menor dimensión.  
El resultado es una matriz con los embeddings reducidos, lista para ser usada en el proceso de agrupamiento.


In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
embeddings_reducidos = pca.fit_transform(np.array(bloques_df['embeddings'].tolist()))


### 7. Agrupamiento de los bloques usando MiniBatchKMeans  
Se agrupan los bloques de texto en clústeres temáticos utilizando el algoritmo **MiniBatchKMeans**.  
- **`n_clusters=5`**: Se especifica que el algoritmo debe crear 5 grupos temáticos.  
- **`batch_size=1000`**: El algoritmo procesa los datos en pequeños lotes de 1000 bloques para mejorar la eficiencia en grandes volúmenes de datos.  
- **`random_state=42`**: Se fija la semilla para garantizar resultados reproducibles.  
Los clústeres asignados a cada bloque se almacenan en la columna `cluster` del DataFrame. Se muestra una vista previa de los primeros bloques con sus respectivos clústeres.


In [17]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 5
kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=1000, random_state=42)
bloques_df['cluster'] = kmeans.fit_predict(embeddings_reducidos)

print(bloques_df[['id', 'guest', 'title', 'cluster']].head())


   id          guest          title  cluster
0   1    Max Tegmark       Life 3.0        1
1   1    Max Tegmark       Life 3.0        1
2   1    Max Tegmark       Life 3.0        1
3   1    Max Tegmark       Life 3.0        1
4   2  Christof Koch  Consciousness        4


E:\InstallApps2023\ANACONDA\Lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


### 8. Selección de bloques representativos por clúster  
Se seleccionan los bloques más representativos dentro de cada clúster basado en la distancia euclidiana al centroide del clúster.  
- **`seleccionar_bloques_representativos(embeddings, centroide, n=3)`**: Calcula la distancia de cada bloque al centroide y selecciona los `n` bloques más cercanos.  
- **`kmeans.cluster_centers_`**: Obtiene los centroides calculados por el algoritmo de agrupamiento.  
- Para cada clúster, se extraen los bloques representativos y se almacenan en un nuevo DataFrame (`representativos_df`).  
Se muestra una vista previa de los bloques representativos seleccionados junto con su `id`, `guest`, `title` y `cluster`.


In [18]:
from sklearn.metrics.pairwise import euclidean_distances

def seleccionar_bloques_representativos(embeddings, centroide, n=3):
    distancias = euclidean_distances(embeddings, [centroide])
    indices_mas_cercanos = np.argsort(distancias.ravel())[:n]
    return indices_mas_cercanos

centroides = kmeans.cluster_centers_

representativos = []
for i in range(n_clusters):
    indices_cl = bloques_df[bloques_df['cluster'] == i].index
    embeddings_cl = np.array(embeddings_reducidos[indices_cl])
    indices_representativos = seleccionar_bloques_representativos(embeddings_cl, centroides[i])
    representativos.extend(bloques_df.iloc[indices_cl].iloc[indices_representativos].to_dict('records'))

representativos_df = pd.DataFrame(representativos)
print(representativos_df[['id', 'guest', 'title', 'bloque_texto', 'cluster']])


     id             guest                                              title  \
0   303        Steve Keen                 Marxism, Capitalism, and Economics   
1   193          Rob Reid  The Existential Threat of Engineered Viruses a...   
2   254  Jay Bhattacharya                         The Case Against Lockdowns   
3   306     Oriol Vinyals  Deep Learning and Artificial General Intelligence   
4   181    Sergey Nazarov    Chainlink, Smart Contracts, and Oracle Networks   
5    44    David Ferrucci  IBM Watson, Jeopardy & Deep Conversations with AI   
6    88    Eric Weinstein  Geometric Unity and the Call for New Ideas, Le...   
7   169         Ryan Hall         Solving Martial Arts from First Principles   
8   259       Thomas Tull  From Batman Dark Knight Trilogy to AI and the ...   
9   223    Travis Stevens               Judo, Olympics, and Mental Toughness   
10  272     Brett Johnson                       US Most Wanted Cybercriminal   
11  125         Ryan Hall  Martial Arts 

### 9. Búsqueda de bloques relevantes basados en una consulta  
Se implementa un sistema de búsqueda semántica para encontrar los bloques más relevantes en función de una consulta dada.  
- **`obtener_embedding_busqueda(clave, modelo)`**: Convierte la consulta de texto en un *embedding* utilizando el modelo Word2Vec.  
- **`cosine_similarity()`**: Calcula la similitud coseno entre el *embedding* de la consulta y los *embeddings* de los bloques de texto.  
- Los valores de similitud se almacenan en la columna `similitudes` del DataFrame.  
- Se seleccionan y ordenan los `top_n` bloques más relevantes y se muestra el resultado con su `id`, `guest`, `title`, texto del bloque, clúster asignado y nivel de similitud.


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

def buscar_bloques_relevantes(clave, bloques_df, modelo, top_n=5):
    embedding_consulta = obtener_embedding_busqueda(clave, modelo)
    embeddings_bloques = np.array(bloques_df['embeddings'].tolist())
    similitudes = cosine_similarity([embedding_consulta], embeddings_bloques)[0]
    
    bloques_df['similitudes'] = similitudes
    indices_top = np.argsort(similitudes)[-top_n:][::-1]
    return bloques_df.iloc[indices_top][['id', 'guest', 'title', 'bloque_texto', 'cluster', 'similitudes']]

# Ejemplo de búsqueda
consulta = "inteligencia artificial"
resultados = buscar_bloques_relevantes(consulta, bloques_df, model)
print(resultados)


       id                guest  \
1833  292         Robin Hanson   
7       3        Steven Pinker   
37     13        Tomaso Poggio   
3       1          Max Tegmark   
33     11  Juergen Schmidhuber   

                                                  title  \
1833  Alien Civilizations, UFOs, and the Future of H...   
7                               AI in the Age of Reason   
37                          Brains, Minds, and Machines   
3                                              Life 3.0   
33             Godel Machines, Meta-Learning, and LSTMs   

                                           bloque_texto  cluster  similitudes  
1833   So we discount the future in terms of caring ...        0     0.276640  
7     You've studied the human mind, cognition, lang...        1     0.268079  
37    The following is a conversation with Tommaso P...        1     0.266727  
3      And that kind of information helps to build t...        1     0.261361  
33     That's the current sort of profit